In [19]:
import json
import os

In [2]:
path = 'qa_dataset_time_sort.json'
qa_dataset = json.load(open(path, 'r', encoding='utf8'))

In [3]:
print("Total question numbers :" + str(len(qa_dataset)))

Total question numbers :24464


In [4]:
filePath ='Domain_qa_dataset/'
indusAndTech = []
for domain, t in [('industrials','train'),('industrials','dev'),('industrials','test'),('technology','train'),('technology','dev'),('technology','test')]:
    file = 'qa_{}_{}.json'.format(domain, t)
    f = json.load(open(os.path.join(filePath, file)))
    indusAndTech += f

In [5]:
print("Indus & Tech question numbers :" + str(len(indusAndTech)))
print("All other domain question number :" + str(len(qa_dataset) - len(indusAndTech)))

Indus & Tech question numbers :5946
All other domain question number :18518


In [46]:
indusAndtech_qid_list = []
for qa in indusAndTech:
    indusAndtech_qid_list.append(qa['qid'])
len(indusAndtech_qid_list)

5946

In [63]:
allOtherDomain = []
for qa in qa_dataset:
    if qa['qid'] not in indusAndtech_qid_list:
        allOtherDomain.append(qa)
len(allOtherDomain)

20674

# -------------------------------------------------------------------------------------
remove duplicate qid in the domain dataset

In [6]:
def read_domain_dataset(domain_Name):
    l = []
    filePath ='Domain_qa_dataset/'
    for domain, t in [(domain_Name,'train'),(domain_Name,'dev'),(domain_Name,'test')]:
        file = 'qa_{}_{}.json'.format(domain, t)
        f = json.load(open(os.path.join(filePath, file)))
        print('file":', file, "len:", len(f))
        l += f
    return l

def to_qid_list(dataset):
    qid_list = []
    for qa in dataset:
        qid_list.append(qa['qid'])
    return qid_list

In [15]:
def remove_duplicate_q(domain): 
    domain_dataset = read_domain_dataset(domain)
    domain_qid_list = to_qid_list(domain_dataset)
    print(len(domain_dataset), len(domain_qid_list), len(set(domain_qid_list)))
    without_duplicate_dataset = []
    qid_included = []
    for q in domain_dataset:
        if q['qid'] not in qid_included:
            without_duplicate_dataset.append(q)
            qid_included.append(q['qid'])
    print(len(without_duplicate_dataset))
    return without_duplicate_dataset

In [16]:
consumer_without_duplicate = remove_duplicate_q('consumer')

file": qa_consumer_train.json len: 2619
file": qa_consumer_dev.json len: 327
file": qa_consumer_test.json len: 328
3274 3274 2155
2155


In [20]:
def domain_train_test_split(domain):
    print(int(len(domain) * 0.8), int(len(domain) * 0.1), len(domain) - int(len(domain) * 0.8)- int(len(domain) * 0.1))
    domain_train = domain[:int(len(domain) * 0.8)]
    domain_dev = domain[int(len(domain) * 0.8):int(len(domain) * 0.8)+int(len(domain) * 0.1)]
    domain_test = domain[int(len(domain) * 0.8)+int(len(domain) * 0.1):]
    print(len(domain_train), len(domain_dev), len(domain_test))
    return domain_train, domain_dev, domain_test

In [ ]:
consumer_train, consumer_dev, consumer_test = domain_train_test_split(consumer_without_duplicate)

In [18]:
def drop_train_dev_test_files(domain, train, dev, test):
    json.dump(train, open('Domain_without_duplicate_dataset/qa_{}_train.json'.format(domain), 'w', encoding='utf8'))
    json.dump(dev, open('Domain_without_duplicate_dataset/qa_{}_dev.json'.format(domain), 'w', encoding='utf8'))
    json.dump(test, open('Domain_without_duplicate_dataset/qa_{}_test.json'.format(domain), 'w', encoding='utf8'))

In [21]:
for domain in ['consumer', 'healthcare', 'industrials', 'technology']:
    domain_without_duplicate = remove_duplicate_q(domain)
    domain_train, domain_dev, domain_test = domain_train_test_split(domain_without_duplicate)
    drop_train_dev_test_files(domain, domain_train, domain_dev, domain_test)

file": qa_consumer_train.json len: 2619
file": qa_consumer_dev.json len: 327
file": qa_consumer_test.json len: 328
3274 3274 2155
2155
1724 215 216
1724 215 216
file": qa_healthcare_train.json len: 1916
file": qa_healthcare_dev.json len: 239
file": qa_healthcare_test.json len: 240
2395 2395 1512
1512
1209 151 152
1209 151 152
file": qa_industrials_train.json len: 2240
file": qa_industrials_dev.json len: 280
file": qa_industrials_test.json len: 281
2801 2801 1823
1823
1458 182 183
1458 182 183
file": qa_technology_train.json len: 2516
file": qa_technology_dev.json len: 314
file": qa_technology_test.json len: 315
3145 3145 1968
1968
1574 196 198
1574 196 198


# ---------------------------------------------------------------------

### Add sentiment template to domain dataset

In [6]:
import json
import numpy as np
import random
import nltk
import svgling
import os
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download([
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
    "maxent_ne_chunker",
    "words",
])

[nltk_data] Downloading package names to /home/yingjie/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yingjie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /home/yingjie/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/yingjie/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/yingjie/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yingjie/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/yingjie/nltk_data.

True

In [8]:
def get_sentiment_word(sentence, polarity='positive'):
    senti = SentimentIntensityAnalyzer()
    senti_score = []
    if polarity=='positive':
        for word in sentence.split():
            senti_score.append(senti.polarity_scores(word)['compound'])
        if max(senti_score) == 0:
            prompt = sentence.split()[random.randint(0, len(sentence.split())-1)]
        else:
            max_index = senti_score.index(max(senti_score))
            prompt = sentence.split()[max_index]
    elif polarity=='negative':
        for word in sentence.split():
            senti_score.append(senti.polarity_scores(word)['compound'])
        if min(senti_score) == 0:
            prompt = sentence.split()[random.randint(0, len(sentence.split())-1)]
        else:
            min_index = senti_score.index(min(senti_score))
            prompt = sentence.split()[min_index]
    else:
        prompt = sentence.split()[random.randint(0, len(sentence.split())-1)]

    return prompt

In [13]:
def generate_sentiment_template_for_domain(qa_dataset):
    senti = SentimentIntensityAnalyzer()
    for qa in qa_dataset:
        query = qa['question']
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')
        for sentence in context_list:
            if answer_text in sentence:
                break

        # sentiment

        query_sentiment = senti.polarity_scores(query)['compound']
        if query_sentiment <= -0.5:
            query_sentiment = 'very negative'
            word = get_sentiment_word(sentence, polarity='negative')
        elif query_sentiment< 0:
            query_sentiment = 'negative'
            word = get_sentiment_word(sentence, polarity='negative')
        elif query_sentiment== 0:
            query_sentiment = 'none'
            word = get_sentiment_word(sentence, polarity='none')
        elif query_sentiment<= 0.5:
            query_sentiment = 'positive'
            word = get_sentiment_word(sentence, polarity='positive')
        else:
            query_sentiment = 'very positive'
            word = get_sentiment_word(sentence, polarity='positive')
        template_02 = 'The sentiment of this question is {}, you need to look for {}'.format(query_sentiment, word)

        template = [template_02,]

        qa['template'] = template
        
    return qa_dataset

### Iterate through all domain files

In [4]:
without_template_path = 'Domain_without_duplicate_dataset/without_template/'
fileList = os.listdir(without_template_path)
fileList

['qa_technology_train.json',
 'qa_industrials_dev.json',
 'qa_industrials_test.json',
 'qa_technology_dev.json',
 '.ipynb_checkpoints',
 'qa_technology_test.json',
 'qa_healthcare_test.json',
 'qa_healthcare_dev.json',
 'qa_consumer_test.json',
 'qa_healthcare_train.json',
 'qa_industrials_train.json',
 'qa_consumer_dev.json',
 'qa_consumer_train.json']

In [21]:
for file in fileList:
    if file[0] != '.':
        domain = file.split('_')[1]
        t = file.split('_')[2][:-5]
        print(domain, t)
        f = json.load(open(os.path.join(without_template_path, file)))
        withSentiment = generate_sentiment_template_for_domain(f)
        json.dump(withSentiment, open('Domain_without_duplicate_dataset/sentiment_template/qa_{}_{}.json'.format(domain, t), 'w', encoding='utf8'))

technology train
industrials dev
industrials test
technology dev
technology test
healthcare test
healthcare dev
consumer test
healthcare train
industrials train
consumer dev
consumer train


# ---------------------------------------------------------------------

### Add phrase template to domain dataset

In [23]:
def generate_phrase_template_for_domain(qa_dataset):
    pattern = """
        NP: {<JJ>+<NN>+}
        NPs: {<JJ>+<NNS>+}
        Ns: {<NN>+<NN>+}
        """
    pattenParse = nltk.RegexpParser(pattern)

    # count=0
    for qa in qa_dataset:
        query = qa['question']
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')

        tokens = nltk.word_tokenize(query)
        tagged = nltk.pos_tag(tokens)
        cs = pattenParse.parse(tagged)
        phrase_template = '"'
        word_template = '"'
        candidates = []
        for i in cs:
            if type(i) != tuple: 
                for word in i:
                    phrase_template += word[0]+' ' 
                phrase_template += ', '
                # break
            elif i[1] == 'JJ' or i[1] == 'NN' or i[1] == 'NNS':
                candidates.append(i[0])
        phrase_template = phrase_template[:-3]+'"'

        if ', ' in phrase_template:
            template_01 = phrase_template + ' are important phrases. '
        elif ' ' in phrase_template:
            template_01 = phrase_template + ' is an important phrase. '
        elif len(phrase_template)>3:
            template_01 = phrase_template + ' is an important word. '
        else:
            template_01 = 'There is no important phrase in this query. '
            # template_01 = ' '

        if len(candidates)>0:
            for candi in range(len(candidates)):
                word_template+=candidates[candi]+'", '
                # if candi >= 2:
                #     break
            word_template = word_template[:-2]

            if template_01 == 'There is no important phrase in this query. ':
                template_02 = 'And also pay attention to these words: ' + word_template
            else:
                template_02 = 'And also pay attention to these words: ' + word_template

            template = [template_01+template_02,]
        else:
            template = [template_01,]

        qa['template'] = template

    return qa_dataset

In [24]:
for file in fileList:
    if file[0] != '.':
        domain = file.split('_')[1]
        t = file.split('_')[2][:-5]
        print(domain, t)
        f = json.load(open(os.path.join(without_template_path, file)))
        withSentiment = generate_phrase_template_for_domain(f)
        json.dump(withSentiment, open('Domain_without_duplicate_dataset/wrong_phrase_template/qa_{}_{}.json'.format(domain, t), 'w', encoding='utf8'))

industrials dev
industrials test
consumer test
industrials train
consumer dev
consumer train
technology train
technology dev
technology test


In [31]:
for file in fileList:
    if file[0] != '.':
        domain = file.split('_')[1]
        t = file.split('_')[2][:-5]
        print(domain, t)
        f = json.load(open(os.path.join(without_template_path, file)))
        withSentiment = generate_phrase_template_for_domain(f)
        json.dump(withSentiment, open('Domain_without_duplicate_dataset/phrase_template/qa_{}_{}.json'.format(domain, t), 'w', encoding='utf8'))

technology train
industrials dev
industrials test
technology dev
technology test
healthcare test
healthcare dev
consumer test
healthcare train
industrials train
consumer dev
consumer train


# ---------------------------------------------------------------------

### Add entity template to domain dataset

In [29]:
import spacy
from spacy import displacy

In [30]:
def generate_entity_template(one_domain_dataset):
    NER = spacy.load('en_core_web_sm')
    for qa in one_domain_dataset:
        template = []
        question = qa['question']
        entities = NER(question).ents
        if len(entities)>0:
            for i in NER(question).ents:
                if i.label_ == 'PERSON':
                    template_01 = 'This person "{}" is menthoned in the question.'.format(i.text)
                    template.append(template_01)
                elif i.label_ == 'GPE':
                    template_01 = 'This country "{}" is menthoned in the question.'.format(i.text)
                    template.append(template_01)
                elif i.label_ == 'ORG':
                    template_01 = 'This organization "{}" is menthoned in the question.'.format(i.text)
                    template.append(template_01)
                elif i.label_ == 'LOC':
                    template_01 = 'This location "{}" is menthoned in the question.'.format(i.text)
                    template.append(template_01)
                elif i.label_ == 'DATE' or i.label_ == 'TIME':
                    template_01 = 'This timing "{}" is menthoned in the question.'.format(i.text)
                    template.append(template_01)
                else:
                    template_01 = 'This entity "{}" is menthoned in the question.'.format(i.text)
                    template.append(template_01)

        else:
            template_00 = 'There is no entity mentioned in the question.'
            template.append(template_00)

        qa['template'] = template
        
    return one_domain_dataset

In [31]:
for file in fileList:
    if file[0] != '.':
        domain = file.split('_')[1]
        t = file.split('_')[2][:-5]
        print(domain, t)
        f = json.load(open(os.path.join(without_template_path, file)))
        withSentiment = generate_entity_template(f)
        json.dump(withSentiment, open('Domain_without_duplicate_dataset/entity_template/qa_{}_{}.json'.format(domain, t), 'w', encoding='utf8'))

industrials dev
industrials test
consumer test
industrials train
consumer dev
consumer train
technology train
technology dev
technology test


# ---------------------------------------------------------------------

### Add Qtype template to domain dataset

#### get top 50 words from only three domains: Indus, tech & consumer

In [40]:
path = 'qa_dataset_time_sort.json'
qa_dataset = json.load(open(path, 'r', encoding='utf8'))

In [41]:
why_list=[]
what_list=[]
whyuseful_list = []
whatwillhappen_list=[]
for qa in qa_dataset:
    query = qa['question']
    (start, end) = qa['answer']
    answer_text = qa['context'][start:end]
    context_list = qa['context'].split('. ')
    for sentence in context_list:
        if answer_text in sentence:
            break
    if "What will happen" in query:
        whatwillhappen_list.append(sentence)
    elif 'What' in query:
        what_list.append(sentence)
    elif ('Why' in query) and ('useful' in query):
        whyuseful_list.append(sentence)
    elif 'Why' in query:
        why_list.append(sentence)
    

In [42]:
def count_words(dataset):
    words = {}
    for i in range(len(dataset)):
        for word in dataset[i].split():
            if word not in words:
                words[word]=1
            else:
                words[word]+=1
    return words

In [43]:
def sort_wordCounter(words_dict):
    # input: a dict of words, key:word, value:times of word shown in the context
    # output: a list sorted by the value of dict
    a = dict(sorted(words_dict.items(), key=lambda x:x[1], reverse=True))
    return a

In [44]:
why_wordcount = sort_wordCounter(count_words(why_list))
what_wordcount = sort_wordCounter(count_words(what_list))
whatwillhappen_wordcount = sort_wordCounter(count_words(whatwillhappen_list))
whyuseful_wordcount = sort_wordCounter(count_words(whyuseful_list))

In [46]:
why_wordcount

{'the': 10134,
 'to': 8200,
 'and': 6776,
 'of': 5766,
 'in': 5347,
 'a': 4071,
 'as': 3187,
 'for': 2214,
 'its': 2211,
 'we': 1993,
 'is': 1879,
 'that': 1828,
 'from': 1683,
 'with': 1556,
 'on': 1468,
 'We': 1325,
 'will': 1295,
 'are': 1271,
 'has': 1197,
 'more': 1070,
 'growth': 1041,
 'be': 1018,
 'which': 1012,
 'it': 1011,
 'by': 984,
 'have': 842,
 'expect': 833,
 'could': 826,
 'revenue': 748,
 'an': 728,
 'our': 707,
 'over': 683,
 'due': 646,
 'or': 645,
 'The': 642,
 'sales': 612,
 'think': 610,
 'at': 598,
 'this': 592,
 'believe': 559,
 'demand': 558,
 'can': 553,
 'market': 551,
 'costs': 540,
 'operating': 530,
 'than': 529,
 'company': 524,
 'cost': 521,
 'new': 519,
 'not': 517,
 'business': 496,
 'also': 479,
 'but': 476,
 'margins': 459,
 'these': 418,
 'margin': 416,
 'should': 413,
 'higher': 404,
 'their': 401,
 'high': 392,
 'would': 391,
 'firm': 382,
 'share': 379,
 'In': 378,
 'risk': 378,
 'lower': 363,
 'customers': 362,
 'products': 359,
 'because': 353

In [47]:
def pmi(wordCount):
    pmi_dict = {}
    for word in wordCount:
        if wordCount[word]<200:
            break
        else:
            num_word = 0
            if word in why_wordcount:
                num_word += why_wordcount[word]
            if word in what_wordcount:
                num_word += what_wordcount[word]
            if word in whyuseful_wordcount:
                num_word += whyuseful_wordcount[word]
            if word in whatwillhappen_wordcount:
                num_word += whatwillhappen_wordcount[word]
            # p_word = num_word / (sum(why_wordcount.values())+sum(what_wordcount.values())+sum(whyuseful_wordcount.values())+sum(whatwillhappen_wordcount.values()))
            # conditional_p_word = wordCount[word] / sum(wordCount.values())
            p_word = num_word / (len(why_wordcount)+len(what_wordcount)+len(whyuseful_wordcount)+len(whatwillhappen_wordcount))
            conditional_p_word = wordCount[word]/len(wordCount)
            
            pmi_dict[word] = np.log(conditional_p_word/p_word)
    
    return pmi_dict

In [48]:
def get_top50_prompt_words(why_wordcount):    
    #input: word count dict
    #output: top 50 prompt word list of corresponds to the word count
    why_word_pmiDict = pmi(why_wordcount) 
    sort_why_pmi = sort_wordCounter(why_word_pmiDict)
    top50_whyPromptWords_list = list(sort_why_pmi)[0:50]
    return top50_whyPromptWords_list

In [49]:
top50_whyPromptWords_list = get_top50_prompt_words(why_wordcount)
top50_whatPromptWords_list = get_top50_prompt_words(what_wordcount)
top50_whyusefulPromptWords_list = get_top50_prompt_words(whyuseful_wordcount)
top50_whatwillhappenPromptWords_list = get_top50_prompt_words(whatwillhappen_wordcount)
print('Why:',len(top50_whyPromptWords_list))
print('What:',len(top50_whatPromptWords_list))
print('Why useful:',len(top50_whyusefulPromptWords_list))
print('What will happen:',len(top50_whatwillhappenPromptWords_list))

Why: 50
What: 50
Why useful: 50
What will happen: 50


In [55]:
top50_whatPromptWords_list

['should',
 'capital',
 'through',
 "company's",
 "firm's",
 'While',
 'competitive',
 'which',
 'some',
 'think',
 'products',
 'The',
 'believe',
 'network',
 'firm',
 'other',
 'its',
 'cost',
 'it',
 'customers',
 'such',
 'strong',
 'has',
 'market',
 'by',
 'also',
 'business',
 'company',
 'can',
 'but',
 'with',
 'product',
 'on',
 'and',
 'In',
 'new',
 'customer',
 'more',
 'that',
 'pricing',
 'their',
 'this',
 'will',
 'the',
 'growth',
 'into',
 'for',
 'next',
 'We',
 'sales']

In [51]:
def generate_Qtype_template(qa_dataset):
    # Random given prompt word for Qtype

    for qa in qa_dataset:
        query = qa['question']
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')
        for sentence in context_list:
            if answer_text in sentence:
                break

    #     # question type

        if "What will happen" in query:
            template_01 = 'To answer a "What will happen" question, you need to look for "{}"'.format(top50_whatwillhappenPromptWords_list[random.randint(0,len(top50_whatwillhappenPromptWords_list)-1)])   
        elif 'What' in query:
            template_01 = 'To answer a "What" question, you need to look for "{}"'.format(top50_whatPromptWords_list[random.randint(0,len(top50_whatPromptWords_list)-1)])   
        elif ('Why' in query) and ('useful' in query):
            template_01 = 'To answer a "Why useful" question, you need to look for "{}"'.format(top50_whyusefulPromptWords_list[random.randint(0,len(top50_whyusefulPromptWords_list)-1)])   
        elif 'Why' in query:
            template_01 = 'To answer a "Why" question, you need to look for "{}"'.format(top50_whyPromptWords_list[random.randint(0,len(top50_whyPromptWords_list)-1)])   

        template = [template_01,]
    
        qa['template'] = template
    
    return qa_dataset

In [56]:
for file in fileList:
    if file[0] != '.':
        domain = file.split('_')[1]
        t = file.split('_')[2][:-5]
        print(domain, t)
        f = json.load(open(os.path.join(without_template_path, file)))
        withSentiment = generate_Qtype_template(f)
        json.dump(withSentiment, open('Domain_without_duplicate_dataset/Qtype_template/qa_{}_{}.json'.format(domain, t), 'w', encoding='utf8'))

technology train
industrials dev
industrials test
technology dev
technology test
healthcare test
healthcare dev
consumer test
healthcare train
industrials train
consumer dev
consumer train


# ------------------------------

# Add empty template

In [25]:
def generate_empty_template_for_domain(qa_dataset):

    for qa in qa_dataset:
    
        qa['template'] = ' '
    
    return qa_dataset

In [32]:
for file in fileList:
    if file[0] != '.':
        domain = file.split('_')[1]
        t = file.split('_')[2][:-5]
        print(domain, t)
        f = json.load(open(os.path.join(without_template_path, file)))
        withSentiment = generate_empty_template_for_domain(f)
        json.dump(withSentiment, open('Domain_without_duplicate_dataset/no_template/qa_{}_{}.json'.format(domain, t), 'w', encoding='utf8'))

industrials dev
industrials test
consumer test
industrials train
consumer dev
consumer train
technology train
technology dev
technology test


In [33]:
75.33-73.26

2.069999999999993

In [34]:
37.66-34.49

3.1699999999999946

In [35]:
61.42-61.28

0.14000000000000057